In [1]:
!pip install tensorflow keras numpy scikit-learn

In [12]:
import numpy as np
import pandas as pd
import os
from google.colab import drive
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [13]:
drive.mount('/content/drive')
FOLDER_PATH = '/content/drive/My Drive/FYP DATAETS'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
file_name = "coin_Bitcoin_processed.csv"
file_path = os.path.join(FOLDER_PATH, file_name)

In [19]:
bitcoin_data = pd.read_csv(file_path, index_col='Date', parse_dates=True)

In [20]:
bitcoin_data.head(10)

,Close,High,Low,Open,Volume,RSI,SMA_50,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9
Date,,,,,,,,,,
2014-11-05,339.485992,343.368988,330.683014,330.683014,19817200.0,37.379565,372.766840,-13.927914,-1.143840,-12.784074
2014-11-06,349.290009,352.966003,338.424011,339.458008,18797000.0,42.486886,370.605960,-12.268993,0.412065,-12.681058
2014-11-07,342.415009,352.731995,341.776001,349.817993,16834200.0,40.021827,368.965460,-11.377884,1.042539,-12.420423
2014-11-08,345.488007,347.032013,342.153992,342.153992,8535470.0,41.651412,367.979301,-10.304920,1.692402,-11.997322
2014-11-09,363.264008,363.626007,344.255005,345.376007,24205600.0,50.097616,367.066501,-7.928815,3.254806,-11.183621
2014-11-10,366.924011,374.816010,357.561005,362.265015,30450100.0,51.649517,366.428561,-5.684871,4.399000,-10.083871
2014-11-11,367.695007,371.309998,363.734985,365.856995,15838900.0,51.988241,365.739421,-3.800507,5.026691,-8.827198
2014-11-12,423.561005,429.717987,367.984985,367.984985,45783200.0,68.957977,365.494821,2.175704,8.802322,-6.626618
2014-11-13,420.734985,457.092987,401.122986,427.273010,58945000.0,67.655286,365.445421,6.607690,10.587446,-3.979756


In [8]:
bitcoin_data.shape

(4045, 10)

In [11]:
close_price_data = bitcoin_data['Close'].values.reshape(-1, 1)
close_price_data

array([[  339.48599243],
       [  349.29000854],
       [  342.41500854],
       ...,
       [90851.7578125 ],
       [90394.3125    ],
       [86726.7890625 ]])

In [28]:
features = [
    'Close', 'High', 'Low', 'Open', 'Volume',
    'RSI', 'MACD_12_26_9', 'MACDs_12_26_9', 'SMA_50'
]

data_array = bitcoin_data[features].values
total_features = data_array.shape[1]
total_rows = len(data_array)


training_size = int(total_rows * 0.8)
print("Total features : ",total_features)
print("Training Size of data : ", training_size ,"days ... (80%)")

Total features :  9
Training Size of data :  3236 days ... (80%)


In [29]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_inputs = scaler.fit_transform(data_array)
train_inputs = scaled_inputs[0:training_size]

In [30]:
import pickle
scaler_save_path = os.path.join(FOLDER_PATH, "bitcoin_scaler.pkl")
with open(scaler_save_path, 'wb') as file:
    pickle.dump(scaler, file)

In [34]:
PREDICTION_DAYS = 60

x_train = []
y_train = []


for i in range(PREDICTION_DAYS, training_size):
    x_train.append(train_inputs[i-PREDICTION_DAYS:i, :])
    y_train.append(train_inputs[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], total_features))

print( "x_train shape : ", x_train.shape)
print( "y_train shape : ", y_train.shape)

x_train shape :  (3176, 60, 9)
y_train shape :  (3176,)


In [40]:
from tensorflow.keras.callbacks import ModelCheckpoint
time_steps = x_train.shape[1]
current_features = total_features
model = Sequential()

model.add(LSTM(units=128, return_sequences=True, input_shape=(time_steps, current_features)))
model.add(Dropout(0.3))
model.add(LSTM(units=128, return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_6 (LSTM)                   │ (None, 60, 128)        │        70,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 60, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 202,369 (790.50 KB)

 Trainable params: 202,369 (790.50 KB)

 Non-trainable params: 0 (0.00 B)

In [45]:
import time
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

checkpoint_path = os.path.join(FOLDER_PATH, "bitcoin_model.keras")
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
model_checkpoint = ModelCheckpoint(
    checkpoint_path,
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)
model.compile(optimizer='adam', loss='mean_squared_error')

print("Starting Model Training")
start_time = time.time()
history = model.fit(
    x_train,
    y_train,
    epochs=100,
    batch_size=32,
    verbose=1,
    validation_split=0.2,
    callbacks=[early_stopping, model_checkpoint]
)

end_time = time.time()
elapsed_time = end_time - start_time
minutes = int(elapsed_time // 60)
seconds = int(elapsed_time % 60)

print(" Model Training Complete.")
print(f"Total training duration: {minutes} minutes and {seconds} seconds.")

Starting Model Training
Epoch 1/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - loss: 1.7250e-04
Epoch 1: val_loss improved from inf to 0.00007, saving model to /content/drive/My Drive/FYP DATAETS/bitcoin_model.keras
80/80 ━━━━━━━━━━━━━━━━━━━━ 12s 122ms/step - loss: 1.7251e-04 - val_loss: 6.6849e-05
Epoch 2/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - loss: 1.5282e-04
Epoch 2: val_loss did not improve from 0.00007
80/80 ━━━━━━━━━━━━━━━━━━━━ 9s 117ms/step - loss: 1.5309e-04 - val_loss: 1.9972e-04
Epoch 3/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - loss: 1.5530e-04
Epoch 3: val_loss did not improve from 0.00007
80/80 ━━━━━━━━━━━━━━━━━━━━ 9s 115ms/step - loss: 1.5533e-04 - val_loss: 9.3439e-05
Epoch 4/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - loss: 1.6320e-04
Epoch 4: val_loss did not improve from 0.00007
80/80 ━━━━━━━━━━━━━━━━━━━━ 9s 111ms/step - loss: 1.6342e-04 - val_loss: 1.1836e-04
Epoch 5/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - loss: 1.4981e-04
Epoch 5: val_loss did 

In [47]:
import pickle
import os
import time
checkpoint_path = os.path.join(FOLDER_PATH, "bitcoin_model.keras")
print(f"Model Saved to: {checkpoint_path}")
scaler_save_path = os.path.join(FOLDER_PATH, "bitcoin_scaler.pkl")
with open(scaler_save_path, 'wb') as file:
    pickle.dump(scaler, file)
print(f"Scaler Saved to: {scaler_save_path}")

Model Saved to: /content/drive/My Drive/FYP DATAETS/bitcoin_model.keras
Scaler Saved to: /content/drive/My Drive/FYP DATAETS/bitcoin_scaler.pkl
